# External Services Lab

### Introduction

In this lesson, we'll use what we learned about ingress services to allow external access to our cluster.  Let's get started.

### Setting it Up

Now, if you have not yet installed the ingress-nginx service,  now is a good time.  Remember that the documentation for doing so is available [here](https://github.com/kubernetes/ingress-nginx/blob/master/docs/deploy/index.md).  

We first run: 

`minikube addons enable ingress`

And then run: 
```bash
kubectl apply -f https://raw.githubusercontent.com/kubernetes/ingress-nginx/controller-v0.43.0/deploy/static/provider/baremetal/deploy.yaml
```

Then confirm that the services are running with:
```bash
kubectl get pods -n ingress-nginx \
  -l app.kubernetes.io/name=ingress-nginx --watch
```

### Connecting our Frontend

Now this time, we would like to allow users to reach our frontend streamlit application.  To do this, we should first create a `frontend-deployment-service` so that we can enable access to any of our client pods.

So let's get started by checking to ensure that this deployment with streamlit exists, then we can create the corresponding service.

> Check the deployments.

> ```bash
NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
api-deployment        2/2     2            2           19h
frontend-deployment   2/2     2            2           20h
```

Now let's confirm that the `api-deployment` already has an ClusterIP service, to allow for internal connections to the relevant pods.  

```bash
kubectl get services
NAME                     TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
api-deployment-service   ClusterIP   10.97.229.151   <none>        5000/TCP   20h
kubernetes               ClusterIP   10.96.0.1       <none>        443/TCP    20h
```

Ok, but now it's time to do the same for the frontend-deployment.

Set the name of the service to `frontend-deployment-service` and make sure that the ports are setup properly to connect to streamlit (check the relevant Dockerfile if you are not sure what port to connect to).

Now let's confirm that we now have *internal* access to the pods hosting streamlit.  Connect to the shell of one of the containers in the api deployment, and then from there make a curl request to streamlit.

> We should see the following.

> <img src="./curl-frontend.png" width="60%">

### Creating the Ingress

Now it's time to create our ingress.  We can write the configuration in the `ingress-service.yaml` file.  And this time, and simply name the service `ingress-service`.  

Make sure that requests are routed to the `frontend-deployment-service` and connect to the correct service port.  To confirm that it is done correctly, find the minikube ip address, and visit the url.  

We should see something like the following:

<img src="./working-application.png" width="40%">

### Adding API access

We may also like to expose our API.  We can do so by adding another path that has any visits to `/api` point to the api service.  So add another path that listens for requests to the following: 

```yaml
- path: /api/?(.*)
```

After setting up the path, we'll need to add one more line to our `annotations` section so that it looks like the following:

```yaml
  annotations:
    kubernetes.io/ingress.class: nginx
    nginx.ingress.kubernetes.io/use-regex: 'true'
    nginx.ingress.kubernetes.io/rewrite-target: /$1
```

The new line will take a request like `/api/restaurants` and change it to `/restaurants`.  In other words, it will only capture the what is after the second `/`.  We add this in so that we can externally make requests to /api, but we will not need to prepend our routes with `/api` in Flask.

If you can visit `/api` and see the corresponding json, your work is done.

### Resources
[Debug Services](https://kubernetes.io/docs/tasks/debug-application-cluster/debug-service/)

[K8s Workflow](https://www.getambassador.io/resources/dev-workflow-intro/)

[Telepresence](https://www.telepresence.io/tutorials/kubernetes-rapid)

